In [10]:
import flask
import json
import pymongo
from flask_cors import CORS;
from flask import request
import re
class Server:
    def createServer(self):
        
            
        try: 
            app = flask.Flask(__name__)
            CORS(app)
            mongoClient = pymongo.MongoClient("mongodb://localhost:27017/")
            db = mongoClient["recipeBook"]
            users = db["users"]
            recipes = db["recipes"]
            
            @app.route("/login",methods = ['POST'])
            def Login():
                loginData = json.loads(request.data)
                #print(loginData)
                user_id = loginData["user_id"]
                password = loginData["password"]
                #print(loginData)
                user = users.find_one({"user_id":user_id , "password":password})
                if(user):
                    return {"data":"valid"}
                else:
                    return {"data":"invalid"}
                
            
            @app.route("/getAllUser")
            def getAllFriends():
                user_id = request.args.get("user_id")
                usersArray = []
                user = users.find_one({"user_id": user_id })
                if(user):
                    usersArray.append({
                            "friendList": user["friend_list"],
                            "friendsTotal":len(user["friend_list"])
                    })
                 
                print(usersArray)
                return{"message":"found", "data":usersArray} 
                
            
            @app.route("/addNewFriend",methods = ['POST'])
            def addNewFriend():
                data = json.loads(request.data)
                print(data)
                user_id = data["user_id"]
                friend_new = data["friendNeedToAdd"]
                user = users.update_one({"user_id":user_id},{
                                       "$push":{"friend_list":friend_new}
                    
                })
                if(user == None):
                    return {"message":"unable to add friend"}
                friend = users.update_one({"user_id":friend_new},
                                          
                                       {"$push":{"friend_list":user_id}})
                if(friend == None):
                    return {"message":"unable to add friend"}
                return {"message":"updated"}
                
            @app.route("/searchFriends")
            def searchFriends():
                user_id = request.args.get("user_id")
                searchedArray = []
                

                rgx = re.compile('.*'+user_id+'.*', re.IGNORECASE)
                user = users.find({"user_id": rgx })
                
                if(user != []):
                    for i in user:
                        
                        searchedArray.append({"user_id":i["user_id"]})
                    print(searchedArray)
                    if(len(searchedArray) != 0):
                        return {"message":"found", "data":searchedArray}
                
                return {"message":"not found"}
            
            
            
            
            @app.route("/getAllRecipes",methods = ['GET'] )
            def fetchAllRecipes():
                user_id = request.args.get("user_id")
                print(user_id)
                user = users.find_one({"user_id":user_id})
                recipe = recipes.find_one({"user_id":user_id})
                if(recipe == None):
                    return {"message":"no recipes found"}
                allRecipes = []
                if(user):
                    userRecipes = recipes.find_one({"user_id": user_id })
                    userRecipesArray = userRecipes["recipes"]
                    if(userRecipesArray != []):
                        for i in userRecipesArray:
                            allRecipes.append(i) 
                    
                    print(allRecipes)
                    friendList = user["friend_list"]
                    print(friendList)
                    if(friendList != []):
                        for i in friendList:
                            friendRecipe = recipes.find_one({"user_id": i })
                            if(friendRecipe == None):
                                continue
                            else:
                                friendRecipesArray = friendRecipe["recipes"]
                            if(friendRecipesArray != []):
                                for j in friendRecipesArray:
                                    allRecipes.append(j)
                            print(allRecipes)
                        if(len(allRecipes)>1):
                            allRecipes.sort(key = lambda x:x["date"],reverse=True)
                            
                #products.sort(key=lambda x: x["brand"])
                    
                      
                    else:
                        print("no friends found")
                    
                    return {"message":"found", "data":allRecipes} 
                 
                return {"message":"no recipes found"}
            
            
            
            @app.route("/addRecipe", methods = ['POST'])
            def addNewRecipe():
                data = json.loads(request.data)
                print(data["user_id"])
                user = recipes.find_one({"user_id":data["user_id"]})
                if(user):
                    userRecipes = recipes.update_one({"user_id": data["user_id"]},{
                        "$push":{
                            "recipes": data
                        }
                    })
                    return {"res":"pushed a new instances of array in recipes table"}
                else:
                    recipes.insert({"user_id": data["user_id"],
                                   "recipes":[data]
                                   })
                    return {"res":"added a new record in recipes table"}
            
            if __name__ == "__main__":
                app.run()
            
        except Exception as e:
            print(e)

In [11]:
s = Server()

In [ ]:
s.createServer()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Oct/2021 21:30:05] "OPTIONS /login HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 21:30:05] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 21:30:05] "GET /getAllRecipes?user_id=rohit HTTP/1.1" 200 -


rohit
[{'name': 'Macaroni', 'user_id': 'rohit', 'ingrediant': '1 (8 ounce) box elbow macaronicup butter,1 cup all-purpose flour,1 teaspoon salt,round black pepper to taste,2 cups milk,2 cups shredded Cheddar cheese', 'steps': 'Melt butter in a saucepan over medium heat; stir in flour, salt, and pepper until smooth, about 5 minutes. Slowly pour milk into butter-flour mixture while continuously stirring until mixture is smooth and bubbling, about 5 minutes. Add Cheddar cheese to milk mixture and stir until cheese is melted, 2 to 4 minutes.', 'date': 1634745581}, {'name': 'Macaroni', 'user_id': 'rohit', 'ingrediant': '1 (8 ounce) box elbow macaronicup butter,1 cup all-purpose flour,1 teaspoon salt,round black pepper to taste,2 cups milk,2 cups shredded Cheddar cheese', 'steps': 'Melt butter in a saucepan over medium heat; stir in flour, salt, and pepper until smooth, about 5 minutes. Slowly pour milk into butter-flour mixture while continuously stirring until mixture is smooth and bubblin

127.0.0.1 - - [20/Oct/2021 21:33:57] "GET /searchFriends?user_id=sahil HTTP/1.1" 200 -


[{'user_id': 'sahil'}]


...

{'user_id': 'rohit', 'friendNeedToAdd': 'sahil'}


127.0.0.1 - - [20/Oct/2021 21:34:01] "GET /getAllRecipes?user_id=rohit HTTP/1.1" 200 -


rohit
[{'name': 'Macaroni', 'user_id': 'rohit', 'ingrediant': '1 (8 ounce) box elbow macaronicup butter,1 cup all-purpose flour,1 teaspoon salt,round black pepper to taste,2 cups milk,2 cups shredded Cheddar cheese', 'steps': 'Melt butter in a saucepan over medium heat; stir in flour, salt, and pepper until smooth, about 5 minutes. Slowly pour milk into butter-flour mixture while continuously stirring until mixture is smooth and bubbling, about 5 minutes. Add Cheddar cheese to milk mixture and stir until cheese is melted, 2 to 4 minutes.', 'date': 1634745581}, {'name': 'Macaroni', 'user_id': 'rohit', 'ingrediant': '1 (8 ounce) box elbow macaronicup butter,1 cup all-purpose flour,1 teaspoon salt,round black pepper to taste,2 cups milk,2 cups shredded Cheddar cheese', 'steps': 'Melt butter in a saucepan over medium heat; stir in flour, salt, and pepper until smooth, about 5 minutes. Slowly pour milk into butter-flour mixture while continuously stirring until mixture is smooth and bubblin

127.0.0.1 - - [20/Oct/2021 22:21:03] "OPTIONS /login HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 22:21:03] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 22:21:03] "GET /getAllRecipes?user_id=sahil HTTP/1.1" 200 -


sahil
[{'name': 'Cheese Pasta', 'user_id': 'sahil', 'ingrediant': '1 (8 ounce) box elbow macaronicup butter,1 cup all-purpose flour,1 teaspoon salt,round black pepper to taste,2 cups milk,2 cups shredded Cheddar cheese', 'steps': 'Melt butter in a saucepan over medium heat stir in flour, salt, and pepper until smooth, about 5 minutes. Slowly pour milk into butter-flour mixture while continuously stirring until mixture is smooth and bubbling, about 5 minutes. Add Cheddar cheese to milk mixture and stir until cheese is melted, 2 to 4 minutes.', 'date': 1634745581}]
['rohit']
[{'name': 'Cheese Pasta', 'user_id': 'sahil', 'ingrediant': '1 (8 ounce) box elbow macaronicup butter,1 cup all-purpose flour,1 teaspoon salt,round black pepper to taste,2 cups milk,2 cups shredded Cheddar cheese', 'steps': 'Melt butter in a saucepan over medium heat stir in flour, salt, and pepper until smooth, about 5 minutes. Slowly pour milk into butter-flour mixture while continuously stirring until mixture is s

In [8]:
import pymongo
from datetime import datetime
class CreateDB:
    def __init__(self):
        mongoClient = pymongo.MongoClient("mongodb://localhost:27017/")
        db = mongoClient["recipeBook"]
        users = db["users"]
        recipes = db["recipes"]
        
        #default login details
        
#         user1 = {"user_id":"rohit", "password":"1234","friend_list":["tom","Holly","harry","anku","Elizabeth"]}
#         user2 = {"user_id":"tom", "password":"4332","friend_list":["rohit"]}
#         user3 = {"user_id":"harry", "password":"86679234","friend_list":["rohit"]}
#         user4 = {"user_id":"anku", "password":"19922","friend_list":["rohit"]}
#         user5 = {"user_id":"Holly ", "password":"2323","friend_list":["rohit"]}
#         user6 = {"user_id":"Carlos ", "password":"5561","friend_list":[]}
#         user7 = {"user_id":"Toni", "password":"124","friend_list":[]}
#         user8 = {"user_id":"Elizabeth ", "password":"123423","friend_list":["rohit"]}
#         user9 = {"user_id":"Ronald ", "password":"34534","friend_list":[]}
#         user10 = {"user_id":"Laverne", "password":"34567","friend_list":[]}
#         user11 = {"user_id":"sahil", "password":"9876","friend_list":[]}
#         users.insert_one(user1)
#         users.insert_one(user2)
#         users.insert_one(user3)
#         users.insert_one(user4)
#         users.insert_one(user5)
#         users.insert_one(user6)
#         users.insert_one(user7)
#         users.insert_one(user8)
#         users.insert_one(user9)
#         users.insert_one(user10)
#         users.insert_one(user11)
#         recipes.insert_one(recipes1)
        
        #default Recipes
        
        recipes1 = {"name":"Macaroni",
                    "user_id":"rohit",
                    "ingrediant":"1 (8 ounce) box elbow macaronicup butter,1 cup all-purpose flour,1 teaspoon salt,round black pepper to taste,2 cups milk,2 cups shredded Cheddar cheese",
                       "steps": "Melt butter in a saucepan over medium heat; stir in flour, salt, and pepper until smooth, about 5 minutes. Slowly pour milk into butter-flour mixture while continuously stirring until mixture is smooth and bubbling, about 5 minutes. Add Cheddar cheese to milk mixture and stir until cheese is melted, 2 to 4 minutes.",
                   "date": int(datetime.now().timestamp())
                   }
        recipes2 = {"name":"Macaroni",
                    "user_id":"rohit",
                    "ingrediant":"1 (8 ounce) box elbow macaronicup butter,1 cup all-purpose flour,1 teaspoon salt,round black pepper to taste,2 cups milk,2 cups shredded Cheddar cheese",
                       "steps": "Melt butter in a saucepan over medium heat; stir in flour, salt, and pepper until smooth, about 5 minutes. Slowly pour milk into butter-flour mixture while continuously stirring until mixture is smooth and bubbling, about 5 minutes. Add Cheddar cheese to milk mixture and stir until cheese is melted, 2 to 4 minutes.",
                   "date": int(datetime.now().timestamp())
                   }
        
        recipes3 = {"name":"Cheese Pasta",
                    "user_id":"sahil",
                    "ingrediant":"1 (8 ounce) box elbow macaronicup butter,1 cup all-purpose flour,1 teaspoon salt,round black pepper to taste,2 cups milk,2 cups shredded Cheddar cheese",
                   "steps": "Melt butter in a saucepan over medium heat stir in flour, salt, and pepper until smooth, about 5 minutes. Slowly pour milk into butter-flour mixture while continuously stirring until mixture is smooth and bubbling, about 5 minutes. Add Cheddar cheese to milk mixture and stir until cheese is melted, 2 to 4 minutes.",
                   "date": int(datetime.now().timestamp())
                   }
        recipes.insert_one({ "user_id":"rohit","recipes":[recipes1,recipes2]})
        recipes.insert_one({"user_id":"sahil","recipes":[recipes3]})
        
        
        
        

In [9]:
db = CreateDB()

In [7]:
from datetime import datetime
int(datetime.now().timestamp())

1634706032

/r/
